In [3]:
# ! pip install fuzzywuzzy
# ! pip install pandas
# ! pip install numpy

In [2]:
import pandas as pd
import numpy as np 
# import datetime

In [5]:
# Read the file into memory
# Assuming the file is named 'simulated_exam_records.csv'
# dtype = str, to ensure all data is read as strings
df = pd.read_csv('data/simulated_exam_records.csv', dtype=str)

In [6]:
# Check the first few rows of the DataFrame
df.head()

,AdmissionNumber,IndexNumber,ExamMonth,ExamYear,DOB_on_rec,CandName_on_rec,CandName_provided,DOB_provided
0,262576,8880,November,2021,2007-10-19,Eric Hopkins,Eric Hopkisn,2007-10-20
1,987434,4249,November,2023,1999-10-08,Michael Bryant,Alyssa Bryant,1999-10-03
2,728162,6614,June,2020,2006-06-12,Erin Weaver DVM,Erin Weaver DVM Christopher,2006-06-12
3,484787,4531,June,2021,2006-02-05,Andrew Jackson,Andrew Jackson,2006-02-10
4,449051,6852,November,2021,2003-03-27,Gregory Diaz,Gregory Diaz,2003-03-22


In [ ]:
# df['DOB_provided'] = pd.to_datetime(df['DOB_provided'], errors='raise',yearfirst=True)

In [ ]:
df.head()

In [ ]:
# Convert 'DOB_provided' to datetime format
# df['DOB_provided'] = df['DOB_provided'].dt.strftime('%d-%m-%Y')

In [ ]:
# Count the number of missing in DOB_provided column
df['DOB_provided'].isna().value_counts()

In [ ]:
df[df['DOB_provided'].isna()]

In [ ]:
df['DOB_on_rec'].dtype

In [ ]:
df['DOB_on_rec']

In [ ]:
# df['DOB'] = df['DOB'].dt.strftime('%d-%m-%Y')

In [ ]:
df.head()

In [ ]:
df[df['DOB_on_rec'].isna()]

In [ ]:
df.info()


## Sorting based on the Status column; whether 
- COMPLETE MIS-MATCH
- 1 COMPLETE MIS-MATCH
- 2 COMPLETE MIS-MATCH
- 3 COMPLETE MIS-MATCH
- 4 COMPLETE MIS-MATCH
- 5 COMPLETE MIS-MATCH
- 6 COMPLETE MIS-MATCH
- COMPLETE MATCH
- COMPLETE MIS-MATCH

### Putting fuzzy ratio functions together to do the column addtion based on the name I specify in the function declaration. 

In [7]:
# Importing the necessary library for fuzzy string matching
# If you don't have fuzzywuzzy installed, you can install it using pip
# !pip install fuzzywuzzy
# A function to calculate the fuzzy ratio between two strings in a column of dataframe
from fuzzywuzzy import fuzz

def calculate_fuzzy_ratio(row):
    return fuzz.ratio(row['CandName_on_rec'], row['CandName_provided'])

In [ ]:
df['FuzzyRatio'] = df.apply(calculate_fuzzy_ratio, axis=1)

In [ ]:
df[['CandName_on_rec', 'CandName_provided', 'FuzzyRatio']].head()

In [8]:
# Function to check similarity between two strings
def calcaulate_set_ratio(row):
    set_ratio = fuzz.token_set_ratio(row['CandName_on_rec'], row['CandName_provided'])
    return set_ratio

In [9]:
df['SetRatio'] = df.apply(calcaulate_set_ratio, axis=1)

In [10]:
df[['CandName_on_rec', 'CandName_provided', 'SetRatio' ]]

,CandName_on_rec,CandName_provided,SetRatio
0,Eric Hopkins,Eric Hopkisn,92
1,Michael Bryant,Alyssa Bryant,67
2,Erin Weaver DVM,Erin Weaver DVM Christopher,100
3,Andrew Jackson,Andrew Jackson,100
4,Gregory Diaz,Gregory Diaz,100
...,...,...,...
995,Ruth Moon DDS,Ruth Moon,100
996,Scott Powell,SCOTT POWELL,100
997,Felicia Coleman,Felicia,100
998,Alexandra Johnston,Alexandra,100


In [11]:
# Function to check similarity between two strings
#from fuzzywuzzy import fuzz
def calculate_sorted_ratio(row):
    sorted_ratio = fuzz.token_sort_ratio(row['CandName_on_rec'], row['CandName_provided'])
    return sorted_ratio

In [12]:
df['SortedRatio'] = df.apply(calculate_sorted_ratio, axis=1)

In [13]:
df[['CandName_on_rec', 'CandName_provided', 'SortedRatio']]

,CandName_on_rec,CandName_provided,SortedRatio
0,Eric Hopkins,Eric Hopkisn,92
1,Michael Bryant,Alyssa Bryant,44
2,Erin Weaver DVM,Erin Weaver DVM Christopher,71
3,Andrew Jackson,Andrew Jackson,100
4,Gregory Diaz,Gregory Diaz,100
...,...,...,...
995,Ruth Moon DDS,Ruth Moon,82
996,Scott Powell,SCOTT POWELL,100
997,Felicia Coleman,Felicia,64
998,Alexandra Johnston,Alexandra,67


In [14]:
df.tail()

,AdmissionNumber,IndexNumber,ExamMonth,ExamYear,DOB_on_rec,CandName_on_rec,CandName_provided,DOB_provided,SetRatio,SortedRatio
995,738701,6620,June,2021,2005-09-19,Ruth Moon DDS,Ruth Moon,2005-09-19,100,82
996,422524,5397,June,2017,2000-04-20,Scott Powell,SCOTT POWELL,2000-04-20,100,100
997,742271,2830,November,2015,2003-03-08,Felicia Coleman,Felicia,2003-03-13,100,64
998,210729,4363,November,2016,2007-08-09,Alexandra Johnston,Alexandra,2007-08-10,100,67
999,953140,8219,June,2017,2008-03-19,Michael Butler,Michael Butler,2008-03-20,100,100


## Feature Engineering 
Conditions contains the if statements, each decision is a tuple and what to print if the decision hold true is contained in the choices list.
The first conditions is that if the the column containing the PercentIfSorted is equal to 100 and the DOB_provided is equal to DOB_on_rec, the choice would be completelty matched. 

In [ ]:
conditions = [
    ((df['SortedRatio'] == 100) & (df['DOB_provided'] == df['DOB_on_rec'])), 
    ((df['SortedRatio']== 100) & (df['DOB_provided'] != df['DOB_on_rec'])), 
    ((df['SortedRatio'] >= 70) & (df['DOB_provided'] == df['DOB_on_rec'])), 
    ((df['SortedRatio'] >= 70) & (df['DOB_provided'] != df['DOB_on_rec'])), 
    ((df['SortedRatio'] >= 60) & (df['DOB_provided'] == df['DOB_on_rec'])), 
    ((df['SortedRatio'] >= 60) & (df['DOB_provided'] != df['DOB_on_rec'])), 
    (df['SortedRatio']<= 59) 
    ]

choices = [
    ('Completely Match'),
    ('Name completely Match but DOB do not'),
    ('Names almost Matches and DOB also Matchess'), 
    ('Names almost matches but DOB do not'), 
    ('One contain the other and DOB matches'), 
    ('One Contains the other and DOB do not match'), 
    ('Do not match') 
    ]

In [ ]:
df['Decision'] = np.select(conditions, choices)

In [ ]:
df[['CandName_on_rec', 'CandName_provided','Decision']]

In [ ]:
# Unpack the unique values into a list
# This will give you a list of unique decisions made in the DataFrame
unique_decision = [* set(df['Decision'])]

In [ ]:
unique_decision

In [ ]:
import re
def clean_sheet_name(name):
    # Replace invalid characters with underscores
    cleaned_name = re.sub(r'[^\w\s]', '_', name)
    return cleaned_name[:31]  # Ensure the name is no longer than 31 characters


# create a excel writer object
# Writing all decision based on the matching to an excel workbook with different sheets of 
with pd.ExcelWriter("workbook.xls") as writer:
    for decision in unique_decision:
        cleaned_name = clean_sheet_name(decision)
        decision_data = df[df['Decision'] == decision]
        decision_data.to_excel(writer, sheet_name=cleaned_name, index=False)

In [ ]:
df.to_excel('checkFuzzy/allwithDOB.xls', index=False)